### 1. Importar data desde API

In [1]:
# Importar CREDENCIALES desed un archivo local
import sys
sys.path.append('../../../')
from OTROS.credentials import api_key, ddbb_user, ddbb_password

In [22]:
# Instalar libreria, solo si no se ha instalado previamente
# pip install alpha_vantage

In [2]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import numpy as np

In [31]:
# Configura el objeto TimeSeries con tu clave de API
ts = TimeSeries(key = api_key)

# Lista de 10 tickers de ejemplo
#tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'FB', 'TSLA', 'NFLX', 'NVDA', 'V', 'PYPL']
tickers = ['AAPL', 'MSFT']

# Crear un DataFrame vacío con las columnas deseadas
df_raw = pd.DataFrame(columns=['ticker', 'date', 'open', 'high', 'low', 'close', 'volume'])

In [32]:
for ticker in tickers:
    data, meta_data = ts.get_daily(symbol=ticker, outputsize='full')
    
    # Guardar los datos en un DataFrame temporal
    df_temp = pd.DataFrame.from_dict(data, orient='index')
    df_temp = df_temp.reset_index()
    df_temp.columns = ['date','open', 'high', 'low', 'close', 'volume']
    df_temp[['open', 'high', 'low', 'close', 'volume']] = df_temp[['open', 'high', 'low', 'close', 'volume']].astype(float)
    
    # Agregar la columna 'Ticker' al DataFrame temporal
    df_temp['ticker'] = ticker
    
    # Concatenar el DataFrame temporal al DataFrame principal
    df_raw = pd.concat([df_raw, df_temp], ignore_index=True)

In [51]:
df_raw.head()

,ticker,date,open,high,low,close,volume
0,AAPL,2023-12-22,195.18,195.41,192.97,193.60,37149570.0
1,AAPL,2023-12-21,196.10,197.08,193.50,194.68,46482549.0
2,AAPL,2023-12-20,196.90,197.68,194.83,194.83,52242815.0
3,AAPL,2023-12-19,196.16,196.95,195.89,196.94,40233138.0
4,AAPL,2023-12-18,196.09,196.63,194.39,195.89,55751861.0


In [53]:
# Dar el formato deseado al dataframe
df = df_raw
df = df.rename(columns={'open':'open_price','high':'high_price','low':'low_price','close':'close_price'})
df['date'] = df['date'].astype(str)
df['ticker'] = df['ticker'].astype(str)

df.head()

,ticker,date,open_price,high_price,low_price,close_price,volume
0,AAPL,2023-12-22,195.18,195.41,192.97,193.60,37149570.0
1,AAPL,2023-12-21,196.10,197.08,193.50,194.68,46482549.0
2,AAPL,2023-12-20,196.90,197.68,194.83,194.83,52242815.0
3,AAPL,2023-12-19,196.16,196.95,195.89,196.94,40233138.0
4,AAPL,2023-12-18,196.09,196.63,194.39,195.89,55751861.0


### 2. Guardar DATAFRAME en una tabla temporal REDSHIFT

In [67]:
# Definir fecha de inicio y fin de exportacion a Redshift
start_date_export = '2023-12-01'
end_date_export = '2023-12-31'

In [68]:
# Filtrar solo la data que se va a actualizar
df_load = df[(df['date']>= start_date_export) 
             & (df['date']<= end_date_export)]

In [69]:
# Validar la data que se enviará hacia REDSHIFT
result = df_load.groupby('ticker').agg(
    fecha_min=('date', 'min'),
    fecha_max=('date', 'max'),
    cant_filas=('date', 'count'),
    cant_dias=('date', 'nunique')
).reset_index()
result

,ticker,fecha_min,fecha_max,cant_filas,cant_dias
0,AAPL,2023-12-01,2023-12-22,16,16
1,MSFT,2023-12-01,2023-12-22,16,16


In [9]:
# Instalar librerias (opcional)
#! pip install psycopg2-binary

In [56]:
# Importar librerias
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [57]:
# Configuración de la conexión a Redshift
redshift_user = ddbb_user
redshift_password = ddbb_password
redshift_host = 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
redshift_port = '5439'
redshift_database = 'data-engineer-database'
redshift_schema = 'gustavo_ortega1929_coderhouse'
redshift_table = 'tabla_temporal'

# Crea el motor de la base de datos para Redshift
engine = create_engine(
    f'postgresql+psycopg2://{redshift_user}:{redshift_password}@{redshift_host}:{redshift_port}/{redshift_database}'
)

# Inserta los datos en la tabla de Redshift
df_load.to_sql(name=redshift_table, con=engine, schema=redshift_schema, if_exists='replace', index=False)

# Cierra la conexión
engine.dispose()

### 3. Cargar la data (UPSERT) desde la tabla temporal hacia tabla productiva

In [58]:
# Importar librerias
from sqlalchemy import text

In [60]:
# Configuración de la conexión a Redshift
redshift_schema_productiva = 'gustavo_ortega1929_coderhouse'
redshift_table_productiva = 'stock_price_data'
redshift_schema_temporal = 'gustavo_ortega1929_coderhouse'
redshift_table_temporal = 'tabla_temporal'

# Define la consulta de upsert
query = text(f"""
    BEGIN;

    -- DELETE de las filas que se van a ACTUALIZAR
    delete from {redshift_schema_productiva}.{redshift_table_productiva}
    where 1=1
        and ticker || date in 
            (select distinct ticker || date 
            from {redshift_schema_temporal}.{redshift_table_temporal}
            )
    ;

    -- INSERT de filas nuevas
    insert into {redshift_schema_productiva}.{redshift_table_productiva}
    select 
        ticker, date, open_price, high_price, low_price, close_price, volume
        , TO_CHAR(CURRENT_TIMESTAMP AT TIME ZONE 'America/Lima', 'YYYY-MM-DD HH24:MI')
    from {redshift_schema_temporal}.{redshift_table_temporal}
    ;

    COMMIT;
""")

# Ejecuta la consulta de upsert
with engine.connect() as connection:
    connection.execute(query)